In [1]:
import torch
from model.make_model import make_model
from config import cfg

In [2]:
cfg.merge_from_file('TIL.yml')
model = make_model(
        cfg,
        num_class=2,
        camera_num=1,
        view_num=1,
        semantic_weight=cfg.MODEL.SEMANTIC_WEIGHT
    )

using Transformer_type: swin_base_patch4_window7_224 as a backbone


C:\Users\alien\Documents\PyCharm-Projects\TIL-2023\CV\SOLIDER-REID\model\backbones\swin_transformer.py:1153: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


===========building transformer===========


In [3]:
model.load_param('log_SGD_continue_1e-4/transformer_29_map0.9159278553764464_acc0.5036038160324097.pth')

Loading pretrained model from log_SGD_continue_1e-4/transformer_29_map0.9159278553764464_acc0.5036038160324097.pth


In [4]:
model.to('cuda')
model.eval()

build_transformer(
  (base): SwinTransformer(
    (patch_embed): PatchEmbed(
      (adap_padding): AdaptivePadding()
      (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (stages): ModuleList(
      (0): SwinBlockSequence(
        (blocks): ModuleList(
          (0-1): 2 x SwinBlock(
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftWindowMSA(
              (w_msa): WindowMSA(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop): DropPath()
            )
            (norm2): LayerNorm((128,), eps=1e-0

In [5]:
model.classifier = torch.nn.Identity()  # remove the classifier layer
model(torch.randn(1, 3, 224, 224).cuda())[0].shape  # model has 2 outputs, global_featmaps, featmaps. We only use global_featmaps

torch.Size([1, 1024])

In [6]:
x = torch.randn(1, 3, 224, 224).cuda()
torch.onnx.export(model, x, 'REID.onnx',  # will produce many small files due to 2GB limit
                  export_params=True,
                  verbose=True,
                  opset_version=18,  # TensorRT 8.6 supports up to 17, Torch 2.0.1 supports up to 18
                  do_constant_folding=True,
                  input_names = ['input'],  # with batch dim and batch dim is query + gallery images, query first. Always at least 2 image input (1 can run but no point cuz we need compare features)
                  output_names = ['global_featmaps', 'featmaps'],  # the feature of all input images. For REID we only use the global featmap
                  dynamic_axes={'input' : {0 : 'batchsize_num_images'}, 'global_featmaps' : {0 : 'batchsize_num_images'}, 'featmaps' : {0 : 'batchsize_num_images'}},  # dynamic batch size
                  keep_initializers_as_inputs=False)

C:\Users\alien\Documents\PyCharm-Projects\TIL-2023\CV\SOLIDER-REID\model\backbones\swin_transformer.py:354: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  output_h = math.ceil(input_h / stride_h)
C:\Users\alien\Documents\PyCharm-Projects\TIL-2023\CV\SOLIDER-REID\model\backbones\swin_transformer.py:355: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  output_w = math.ceil(input_w / stride_w)
C:\Users\alien\Documents\PyCharm-Projects\TIL-2023\CV\SOLIDER-REID\model\backbones\swin_transformer.py:356: TracerWarning: Converting a tensor to a Python boolean might cau

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
import onnxruntime as ort
import cv2
from PIL import Image
import numpy as np
from config import cfg
import torch

cfg.merge_from_file('TIL.yml')

sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

ort.get_available_providers()

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [8]:
sess = ort.InferenceSession("REID.onnx", sess_options=sess_options, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [9]:
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.transpose(img, (2, 0, 1))
    # normalize with mean and std supplied in cfg
    img = img / 255.0
    for channel in range(3):
        img[channel] -= cfg.INPUT.PIXEL_MEAN[channel]
        img[channel] /= cfg.INPUT.PIXEL_STD[channel]
    return img.astype(np.float32)

query_paths = ["../RT-DETR/dataset/reid/test_old/query/image_0000.png"]
gallery_paths = ["../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_1.png", "../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_2.png"]
imgs = [load_img(path) for path in query_paths + gallery_paths]
img = np.stack(imgs, axis=0)  # stack the query and gallery images as batch dim
output_dict = ["global_featmaps", "featmaps"]
inputs_dict = {'input': img}

# result = model(torch.from_numpy(img).to('cuda'))[0].cpu().detach().numpy()
result = sess.run(output_dict, inputs_dict)[0]  # only the first output aka global_featmaps is used for REID

In [10]:
result.shape

(3, 1024)

In [11]:
from utils.metrics import Postprocessor

num_query = len(query_paths)  # each image only has 1 suspect given but if we doing rotation then maybe 4
postprocessor = Postprocessor(num_query=num_query, max_rank=50, feat_norm=cfg.TEST.FEAT_NORM, reranking=False)  # in finals cannot use RR as threshold will be changed based on gallery size

postprocessor.update(torch.from_numpy(result).to('cuda'))  # postprocessor expects Torch tensor as it uses torch to compute stuff
dist_mat = postprocessor.compute()
# perform thresholding to determine which gallery image, if any, are matches with the query
# dist_mat = (dist_mat < cfg.TEST.THRESHOLD).astype(int)  # boolean array
results = []
for i, path in enumerate(gallery_paths):
    results.append((path, dist_mat[0][i]))
results

The test feature is normalized
=> Computing DistMat with euclidean_distance


C:\Users\alien\Documents\PyCharm-Projects\TIL-2023\CV\SOLIDER-REID\utils\metrics.py:11: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1485.)
  dist_mat.addmm_(1, -2, qf, gf.t())


[('../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_1.png',
  0.0016824007),
 ('../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_2.png',
  0.002221346)]